<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter3-FINE-TUNING-PERETRAINED_MODEL/section2-Fine-tuning-a-model-with-the-Trainer-API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a model with the Trainer API

<div dir="rtl">
  <p>
    کتابخانه <strong>Transformers</strong> کلاس <code>Trainer</code> را برای کمک به <strong>fine-tune</strong> مدل‌های پیش‌آموزش‌دیده فراهم می‌کند. پس از انجام تمام مراحل پیش‌پردازش داده‌ها، تنها چند مرحله باقی مانده برای تعریف <code>Trainer</code>.
  </p>
  <p>
    یکی از چالش‌ها سرعت کند اجرای <code>Trainer.train()</code> در <strong>CPU</strong> است که بهتر است از <strong>GPU</strong> استفاده شود.
  </p>
</div>


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

<div dir="rtl">
  <p>
    کد زیر در بخش قبلی اجرایی و توضیح داده شد:
  </p>
</div>


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

<div dir="rtl">
  <p>
    <strong>آموزش</strong>
  </p>
  <p>
    برای شروع آموزش، ابتدا باید کلاس <code>TrainingArguments</code> را تعریف کنید که هایپرپارامترهای مورد نیاز برای آموزش و ارزیابی را شامل می‌شود. تنها آرگومانی که باید مشخص کنید، دایرکتوری ذخیره مدل و چک‌پوینت‌ها است، و برای باقی موارد می‌توانید از مقادیر پیش‌فرض استفاده کنید.
  </p>
</div>


In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

<div dir="rtl">
  <p>
    مرحله دوم، تعریف مدل است. مانند فصل قبلی، از کلاس <strong>AutoModelForSequenceClassification</strong> استفاده خواهیم کرد، با دو برچسب:
  </p>
</div>


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

<div dir="rtl">
  <p>
    پس از ساخت مدل پیش‌آموزش‌دیده، هشدارهایی دریافت می‌شود زیرا هد مدل برای طبقه‌بندی جفت جملات تغییر کرده است. سپس، مدل، هایپرپارامترها، مجموعه داده‌ها، <code>data_collator</code> و <code>processing_class</code> را به <code>Trainer</code> می‌دهیم. پارامتر <code>processing_class</code> به <code>Trainer</code> می‌گوید از کدام توکنایزر برای پردازش داده‌ها استفاده کند.
  </p>
</div>


In [6]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

<div dir="rtl">
  <p>
    زمانی که توکنایزر را به‌عنوان <code>processing_class</code> به <code>Trainer</code> بدهید، <code>DataCollatorWithPadding</code> به‌طور پیش‌فرض به‌عنوان <code>data_collator</code> استفاده می‌شود. در این صورت، می‌توانید خط مربوط به <code>data_collator</code> را حذف کنید.
  </p>
</div>


---



<div dir="rtl">
  <p>
    برای <strong>fine-tune</strong> کردن مدل روی مجموعه داده خود، کافی است که متد <code>train()</code> را از <code>Trainer</code> فراخوانی کنیم:
  </p>
</div>


In [ ]:
trainer.train()

<div dir="rtl">
  <p>
    <strong>ارزیابی</strong>
  </p>
  <p>
    برای ارزیابی مدل، باید تابع <code>compute_metrics()</code> را تعریف کنیم که یک شیء <code>EvalPrediction</code> می‌گیرد (یک تاپل نام‌دار با فیلدهای <code>predictions</code> و <code>label_ids</code> است) و یک دیکشنری از معیارهای ارزیابی را به‌عنوان خروجی برمی‌گرداند.
  </p>
  <p>
    برای دریافت پیش‌بینی‌ها از مدل، از دستور <code>Trainer.predict()</code> استفاده می‌کنیم.
  </p>
</div>


In [8]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


<div dir="rtl">
  <p>
    خروجی متد <code>predict()</code> یک تاپل نام‌دار با سه فیلد است:
  </p>
  <ul>
    <li><strong>predictions:</strong> پیش‌بینی‌های مدل به صورت آرایه‌ای دو بعدی.</li>
    <li><strong>label_ids:</strong> شناسه‌های برچسب‌های واقعی.</li>
    <li><strong>metrics:</strong> معیارهای ارزیابی مانند خطا و زمان پیش‌بینی.</li>
  </ul>
  <p>
    برای تبدیل پیش‌بینی‌ها به نتایج نهایی، باید بیشترین مقدار از هر ردیف آرایه <code>predictions</code> را انتخاب کرد که نمایانگر کلاس پیش‌بینی‌شده است. اگر تابع <code>compute_metrics()</code> تعریف شود، معیارهای اضافی مانند دقت یا F1 نیز محاسبه می‌شوند و به فیلد <code>metrics</code> اضافه می‌شوند.
  </p>
</div>

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

<div dir="rtl">
  <p>
    برای مقایسه <code>predictions</code> با <code>labels</code> و ساخت تابع <code>compute_metrics()</code>، از معیارهای کتابخانه هاگینگ فیس <strong>Evaluate</strong> استفاده می‌کنیم. با استفاده از تابع <code>evaluate.load()</code> می‌توانیم معیارهای مربوط به مجموعه داده MRPC را بارگذاری کرده و از متد <code>compute()</code> برای محاسبه معیارها استفاده کنیم.
  </p>
</div>

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()